In [1]:
#import files & libraries
import os
import pandas as pd
cwd = os.path.abspath('') 
files = os.listdir(cwd)
files

['.ipynb_checkpoints',
 'Input_Q1 2021 _Company A.xlsx',
 'Input_Q1 2021 _Company B.xlsx',
 'Input_Q1 2021 _Company C.xlsx',
 'Input_Q1 2021 _Company D.xlsx',
 'Input_Q1 2021 _Company E.xlsx',
 'Task1_DataAnalysis-Test_APap.ipynb']

In [2]:
#create and clean dataframe
dataframes = []

for file in files:
     if file.endswith('.xlsx'):
        df = pd.read_excel(file)
        #add company to differentiate between product stats
        df["Company"]= file[23]
        dataframes.append(df)   

df = pd.concat(dataframes)
df.Product.replace({"Product ":""}, regex = True, inplace = True)
df.head()

,Family,Product,Country 1,Country 2,Country 3,Country 4,Country 5,Country 6,Company
0,1,1.1,0,0,0,0,0,0,A
1,1,1.2,0,0,0,0,0,0,A
2,1,1.3,411,175,67,67,67,0,A
3,1,1.4,1520,67,68,67,67,0,A
4,2,2.1,489,199,67,67,106,0,A


In [3]:
product_list = df.Product.unique().tolist()
country_list = ["Country 1", "Country 2", "Country 3", "Country 4", "Country 5", "Country 6"]

In [4]:
#create a dataframe for aggregates
total_df = df.groupby(["Family", "Product"]).sum()
total_df = total_df.reset_index()

#add a row with overall total
total = df[country_list].sum()
total.name = "Total"
total_df = total_df.append(total.transpose())

total_df

,Family,Product,Country 1,Country 2,Country 3,Country 4,Country 5,Country 6
0,1.0,1.1,1363.0,15571.0,164.0,29983.0,1040.0,49.0
1,1.0,1.2,514.0,11811.0,141.0,22203.0,371.0,54.0
2,1.0,1.3,1768.0,43163.0,989.0,8291.0,2064.0,310.0
3,1.0,1.4,8055.0,24001.0,688.0,32212.0,861.0,269.0
4,2.0,2.1,2262.0,9242.0,202.0,1567.0,1037.0,298.0
5,2.0,2.2,502.0,119.0,84.0,844.0,283.0,54.0
6,2.0,2.3,1999.0,34954.0,323.0,34841.0,2476.0,64.0
7,2.0,2.4,1167.0,15123.0,271.0,3952.0,1298.0,160.0
Total,NaN,NaN,17630.0,153984.0,2862.0,133893.0,9430.0,1258.0


In [5]:
#if the product value is higher than the total 80% of the product in that country
#replace the total value with "Conf"
for n in country_list:
    for i in product_list:
        new_df = df[(df["Product"] == i) & (df[n]> 0.8*int(total_df.loc[total_df["Product"] == i, n]))]
        if len(new_df.index) > 0:
            total_df.loc[total_df["Product"] == i, n] = "Conf"

In [6]:
for n in country_list:
#if the product value is higher than 0, add 1 to new variable
    for i in product_list:
        filtered_df = df[(df["Product"] == i) & (df[n] > 0)]
        new_var = len(filtered_df.index)
    
#if variable length is less than 3, replace product in total_df with "Conf"
        if new_var < 3:
            total_df.loc[(total_df["Product"] == i), n] = "Conf"

total_df

,Family,Product,Country 1,Country 2,Country 3,Country 4,Country 5,Country 6
0,1.0,1.1,1363,15571,164.0,29983,1040,Conf
1,1.0,1.2,Conf,Conf,141.0,22203,Conf,Conf
2,1.0,1.3,1768,43163,989.0,Conf,2064,Conf
3,1.0,1.4,8055,24001,688.0,Conf,861,Conf
4,2.0,2.1,2262,9242,202.0,1567,1037,Conf
5,2.0,2.2,502,119,84.0,844,283,Conf
6,2.0,2.3,1999,34954,323.0,34841,2476,Conf
7,2.0,2.4,1167,15123,271.0,3952,1298,Conf
Total,NaN,NaN,17630,153984,2862.0,133893,9430,1258


In [7]:
#export to Excel
total_df.to_excel("Task1_PythonResults_raw-Test_APap.xlsx")